In [246]:
# -*- coding: utf-8 -*- 
import urllib , requests , sys ,string ,time
from bs4 import BeautifulSoup
from bs4 import SoupStrainer
from time import gmtime, strftime

#doujinshi
#fkey作者來源key.txt
#fcomiclist結果key_doujinshi.text

#輸出格式：
#dmm
#!作者
#!總筆數
#==類別_數量_中文敘述
#[同人團_作者群][原作][同人場cXX]同人作品
#參考網址

#頁顯示數量
pnum = 50
mlink = 'http://www.doujinshi.org/'

fkey = open('key.txt', 'r')
key = fkey.readline()#key=作者
fkey.close()

#key=作者
key2 = urllib.quote(key.decode('utf8').encode('shift_jis'))#當sjis輸出utf8的url
key3 = urllib.unquote(key2.decode('shift_jis').encode('utf8'))#utf8的url翻sjis
#網址用
key4 = urllib.quote(key)

#檢查BOM
if '%EF%BB%BF' in urllib.quote(key):
    print 'fuck ms'

'''搜尋網址
http://www.doujinshi.org/search/item/?order=objects&T=author&flow=DESC&sn=%E7%8A%AC&Q=s&match=3&page=1
http://www.doujinshi.org/search/item/
?Q=s&
T=author&#搜尋項目
sn=%E7%8A%AC&#關鍵字
match=3&#搜尋模式_0模糊1前2後3精確4聲似
order=objects&#排序
flow=DESC#升降_ASC升 DESC降
'''
#page=1
link="http://www.doujinshi.org/search/item/\
?order=objects&T=author&flow=DESC&sn="+key4+"&Q=s&match=3&page=1"
#match=3_match=1
#搜尋例_東山翔_犬_inu
#資料缺少例_方言
#馬甲例_大友卓二

#cookis_語言預設英文，透過cookie設定。
cookies = {'PHPSESSID':'qlac2h749s2fpjtgqlnmqbvlo4',
           ' AGE':'18',
           '__utmt':'1',
           '__utma':'221679373.1195927157.1458231980.1458231980.1458231980.1',
           '__utmb':'221679373.6.10.1458231980',
           '__utmc':'221679373',
           '__utmz':'221679373.1458231980.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none)',
           'LANG':'2'
}

res = requests.get(link,cookies=cookies)
#res = requests.get(link)
res.encoding =  res.apparent_encoding#亂碼處理
#r= res.text
#r=r[r.find(u'<td id="mu">'):r.find(u'<!-- /mu --></td>')+len(u'<!-- /mu --></td>')]#+'</td>'#搜尋結果擷取

#only_a_tags = SoupStrainer(id="mu")#縮小檢索範圍
soup = BeautifulSoup(res.text,"html.parser")#,  parse_only=only_a_tags)

#全轉半_unicode輸入
def Q2B(ustring):
    fs=u'０１２３４５６７８９ＱｑＷｗＥｅＲｒＴｔＹｙＵｕＩｉＯｏＰｐＡａＳｓＤｄＦｆＧｇＨｈＪｊＫｋＬｌＺｚＸｘＣｃＶｖＢｂＮｎＭｍ'
    hs=u'0123456789QqWwEeRrTtYyUuIiOoPpAaSsDdFfGgHhJjKkLlZzXxCcVvBbNnMm'
    rstr = ''
    for tm in ustring:
        if fs.find(tm)+1:
            tm = hs[fs.find(tm)]
        rstr = rstr + tm
    ustring = rstr
    ustring=ustring.lower()
    return ustring
#半轉全
def B2Q(ustring):
    fs=u'０１２３４５６７８９ＱｑＷｗＥｅＲｒＴｔＹｙＵｕＩｉＯｏＰｐＡａＳｓＤｄＦｆＧｇＨｈＪｊＫｋＬｌＺｚＸｘＣｃＶｖＢｂＮｎＭｍ'
    hs=u'0123456789QqWwEeRrTtYyUuIiOoPpAaSsDdFfGgHhJjKkLlZzXxCcVvBbNnMm'
    rstr = ''
    ustring=ustring.lower()
    for tm in ustring:
        if hs.find(tm)+1:
            tm = fs[hs.find(tm)]
        rstr = rstr + tm
    ustring = rstr
    return ustring

#資料儲存
def save(sdict , check=0):
    for tep in listdata:
        if sdict.get(tep):
            #fout.write(sdict[tep].encode('utf8')  + '\n')
            fout.write(sdict[tep]  + '\n')
    #return

#作品頁面
def finddata(dlink,check):#,oname):
    global oname
    res = requests.get(dlink,cookies=cookies)
    res.encoding =  res.apparent_encoding#亂碼處理
    rr= res.text
    rtn=rr[rr.find(u'<tr><td><B>イベント:'):rr.find(u'<tr><td><B>シリーズ:')]
    rr=rr[rr.find(u'<tr><td colspan=3><br><div class="list">著者:'):]
    rr=rr[:rr.find(u'<tr><td colspan=3><br><div class="list">',1)]
    
    if check==1:
        rr=rtn+'\n'+rr
    so=BeautifulSoup(rr,"lxml")
    
    dcheck=0
    s=0
    dcxx=''
    dname=''
    doname=''
    n=0
    
    #cxx
    if check==1:
        dcxx=so.a.text
        s=2
    
    #作者
    for b in so.select('td')[1+s::3]:#日文_英文_空
        #print Q2B(b.text).strip().strip('\n').replace('\t','').replace('\n','')
        #print len(oname),oname
        if Q2B(b.text).strip().strip('\n').replace('\t','').replace('\n','')==key.decode('utf8'):
            dcheck=1
        elif (len(oname)>0) and (Q2B(b.text).strip().strip('\n').replace('\t','').replace('\n','') in oname):
            doname=Q2B(b.text).strip().strip('\n').replace('\t','').replace('\n','')
            #print doname,Q2B(b.text).strip().strip('\n').replace('\t','').replace('\n','')
        dname=dname+Q2B(b.text).strip().strip('\n').replace('\t','').replace('\n','')+'_'
        n=n+1
    dname=dname.strip('_')
    #print dname,n
    
    #類型
    if (n>1) and (check == 1):#非雜誌的多作者
        check=2
    elif (n>1) and (check != 4):#同人的多作者
            check=5
    if (dcheck==0) and (check==2 or check==1):#同人馬甲
        check=6
        #if check==2 or check==1:#馬甲
    elif dcheck==0:
        check=7
        #print doname
    #name=
    #cxx
    
    #book_club_raw
    return dname,dcxx,check,doname,n

#資料處理
def findbook(page = 1, link = 'www'):
    global mindate,oname
#def findbook(soup , page = 1):
    if len(link)>5:
        print 'get'
    link=link+'?order=title&page='+str(page)
    #http://www.doujinshi.org/browse/author/42893/Hisasi/?order=title&page=1
    
    res = requests.get(link,cookies=cookies)#日文介面
    res.encoding =  res.apparent_encoding
    r= res.text
    r1=r[r.find(u'<div class="rounded_box">'):r.find(u'<!--rounded_box-->')+len(u'<!--rounded_box-->')]#別名內容
    rt='<table border=0 width=\"100%\" class=\'tablefixing\'>'
    r2=r[r.rfind(rt)+len(rt):r.rfind('</table>')]#+'</td>'#主內容
    
    
    #oname
    if (page==1) and (u'別名' in r1):
        so = BeautifulSoup(r1,"lxml")
        #if u'別名' in r1:
        
        rt=r1[r1.find(u'日本名:</B>')+len(u'日本名:</B>'):r1.find(u'<B>名前')]
        rt=rt.replace('<br>','').strip().strip('\n').replace('\t','')
        r1=r1[r1.find(u'別名:</B><br>')+len(u'別名:</B><br>'):r1.find(u'<br><B>所属')]
        r1=r1.replace('</A><br>','')
        oname=r1.strip().strip('\n').replace('\t','').split('\n')
        print oname
        tt=0
        for ttemp in oname:
            oname[tt]=ttemp.strip().strip('\n')
            tt=tt+1
        oname.append(rt)
        print oname
    
    bn=r2.count('\"thumbnail\"')
    doc=r2.split('<!--round_middle-->')
    doc.pop()
    print bn,len(doc)#,doc[0],doc[50]
    
    a =0
    for tm in doc[:]:
        sou = BeautifulSoup(tm,"lxml")
    
        cbook=sou.select('.tab.LPEXACT1')[0].text
        cclub=sou.select('.tab.LPEXACT1')[2].text
        craw=sou.select('.tab.LPEXACT1')[4].text
        ctype=sou.select('.tab.LPEXACT1')[5].text
        
        clink=sou.a.get('href')
        cdata=clink[6:]
        cdata=cdata[:cdata.find('/')]
        cdate=sou.select('.tab.L33')[1].text
        cdate=cdate[cdate.find(u'発行日:')+len(u'発行日:'):cdate.find(u'-')].strip().strip('\n')
        
        a=a+1
        #print '\r',a,
        
        blink=mlink+clink[1:]
        #print blink,
        
        #類型
        if (u'同人CG' in ctype) or (u'同人ノベル' in ctype) or (u'同人誌' in ctype):
            check=1
        elif u'漫画' in ctype:
            check=3
        elif u'雑誌' in ctype:
            check=4
        elif (u'商業ノベル' in ctype) or (u'商業本その他' in ctype):
            check=5
        else:
            #a=a+1
            print 'X'
            continue
        
        #作者與場次
        cname,ccxx,check,coname,n=finddata(blink,check)#,oname)#連結_別名_判斷類型
        #club_http://www.doujinshi.org/book/643590/Suisei-no-Garugantia-Illust-Fan-Book/
        
        #print cdata,cbook,cclub,craw,ctype,cdate,coname,n#,clink
        #print 'check:',check,
        #cname
        #
        
        #日期
        if ((check==4) or (check==3)) and (int(cdate) > 1000) and (mindate - int(cdate) > 0):
            mindate=int(cdate)
            #print mindate,
        
        
        #data處理
        if len(cdata) < 3:#無data
            cdata = '0000/00/00'#填入data
            check = 4#
        data = cdata
        while listdata.count(data):#重複data判斷
            data = data[:-2] + str(int(data[-2:]) + 1).rjust(2,'0')#尾數+1_十位數填0
        listdata.append(data)
        #print data
        
        ###
        
        print '\r',a,'check',check,blink
        
        ctype=ctype.encode('utf8')
        cbook=cbook.encode('utf8')
        craw=craw.encode('utf8')
        cclub=cclub.encode('utf8')
        coname=coname.encode('utf8')
        cdate=cdate.encode('utf8')
        cname=cname.encode('utf8')
        ccxx=ccxx.encode('utf8')
        #print type(craw),type(cclub),type(ccxx),type(coname),type(cdata),type(cdate),type(blink),type(cname),
        #<type 'unicode'> <type 'unicode'> <type 'str'> <type 'unicode'> <type 'str'> <type 'unicode'>
        if n>5 :#太多作者
            cname=key
            if check > 5:#馬甲
                #print coname
                cname=coname
        
        #寫入dict
        
        if check==1:#
            book='['+craw+']['+ccxx+']'+cbook
            dict1.setdefault(data,book)
        elif check==2:#
            book='['+cclub+']['+cname+']['+craw+']['+ccxx+']'+cbook+'\n!'+str(n)+'!'+blink
            dict2.setdefault(data,book)
        elif check==3:#
            book=cbook
            dict3.setdefault(data,book)
        elif check==4:#
            book='['+cname+']'+cbook+'\n!'+str(n)+'!'+blink
            dict4.setdefault(data,book)
        elif check==5:#
            book='['+ctype+']'+'['+cname+']'+cbook+'\n!'+str(n)+'!'+blink
            dict5.setdefault(data,book)
        elif check==6:#
            book='['+ctype+']['+cclub+']['+cname+']'+'['+craw+']['+ccxx+']'+cbook+'\n!'+str(n)+'!'+blink
            dict6.setdefault(data,book)
        else:#
            book='['+ctype+']'+'['+cname+']'+cbook+'\n!'+str(n)+'!'+blink
            dict7.setdefault(data,book)
        
        print '#'+book
        #club=cclub.strip().strip(u'\n').replace('\n','')
        #if a>4:
        if a==(bn):
            break
        continue
        #continue
    print '.'
    #print '========'
    #return

########START
'''
商業アートブック_商業CG_商業その他_商業ソフト_漫画 (抜粋)
海賊版_カレンダー_ムック_ポストカード_ポスター_下敷き_テレホンカード_不詳
同人グッズ_同人映像作品_同人音楽作品_同人その他_同人ソフト

同人CG_同人ノベル_同人誌
漫画
雑誌
商業ノベル_商業本その他

1單同_同人CG_同人ノベル_同人誌
2多同

3單漫_漫画
4雑誌
5多漫_商業ノベル_商業本その他

6別名同
7別商
#'''

if len(soup.select('.next'))>0:#存在下一頁
    mp=len(soup.select('.header')[0].text)-2
    fplink=link[:-1]+str(mp)
    print 'Big~'

print soup.select('.header')[0].text#,soup.select('.next')[0]#,link

key=key.lower()
pn=0
nname=0
a=0
c=0
if len(soup.find_all(title='More Info'))>0:
    print 'OK'

for temp in soup.find_all(title='More Info'):
    
    tmp=str(temp)
    tmp=tmp[:tmp.rfind('<br>')]
    #print tmp
    tmp=tmp[tmp.find('"More Info">')+len('"More Info">'):tmp.rfind('(')].strip('\t\n ').lower()
    #print temp.get('href'),'_',tmp,key
    if key == tmp:
        nname=nname+1
        #print tmp,len(tmp)
        if c==0:
            c=1
            nlink=temp.get('href')
            nid=nlink[15:]
            nid=nid[:nid.find('/')]
            #/browse/author/44084/Sano-Toshihide/
            nlink=mlink+nlink[1:]
            print nlink,nid,
            #print "!!!!",
            pt=soup.find_all(title='More Info')[a].parent.parent.td.a.text
            pn=int(pt[:pt.find(u'作品')])#資料筆數
            #print '#',pn,
    a=a+1
    #print '.'

print '符合筆數'+str(nname)+'_作品筆數'+str(pn)

#資料筆數_是否數字
if pn>0:
    if pn > pnum:
        print 'BIG'
    
    fout = open('output/'+key.decode('utf8') + '_doujishiv1.txt', 'w')#寫入模式開檔
    fout.write('doujishi\n')#doujishi
    print key , pn , 'num\n========v1'
    time.sleep(1)
    fout.write('!' + key + '\n!總筆數' + str(pn).encode('utf8') +'_'+ strftime("%Y/%m/%d,%H:%M")+'->')
    
    p = 0#頁
    #建空輸出用字典與陣列
    dict1={}#同人目標作者
    dict2={}#同人多作者
    dict3={}#漫畫
    dict4={}#雜誌
    dict5={}#其他商業
    dict6={}#馬甲同人
    dict7={}#馬甲商業
    listdata = []
    mindate=9999#出道年
    oname=[]#別名
    
    #資料處理
    while (int(pn) - p * pnum) > 0:
        p = p + 1
        print 'page:' + str(p)
        #soup = next(p, nlink)#頁
        
        #print soup.select('.TBLdtil')[2]
        #findbook(soup)#資料處理
        findbook(p, nlink)#資料處理
        time.sleep(1)
        
    #sys.exit()################
    #日期排序
    #listdata.sort()
    
    fout.write(strftime("%H:%M")+'\n')
    fout.write(str(mindate))
    for tm in oname:
        fout.write(tm.encode('utf8')+'_')
    fout.write('\n')
    
    #dict1_同人
    fout.write('==book_' + str(len(dict1)) +'_單行本\n')
    save(dict1)
    #dict2_
    fout.write('==sbook_' + str(len(dict2)) +'_單話\n')
    save(dict2)
    #dict3_漫畫
    fout.write('==other_' + str(len(dict3)) +'_多作者\n')
    save(dict3)
    #dict4_雜誌
    fout.write('==other_' + str(len(dict4)) +'_多作者\n')
    save(dict4)
    #dict5_其他商業
    fout.write('==other_' + str(len(dict5)) +'_多作者\n')
    save(dict5)
    #dict6_馬甲同人
    fout.write('==other_' + str(len(dict6)) +'_多作者\n')
    save(dict6)
    #dict7_馬甲商業
    fout.write('==other_' + str(len(dict7)) +'_多作者\n')
    save(dict7)
    
    #sys.exit()################
    fout.close()
    print 'ok'
elif pn:
    print 'No Date'

#結束讀秒
x=3
while x!=0:
    print x,'..',
    x=x-1
    time.sleep(1)
raw_input("\nPress Any Key To Exit")

◄1►
OK
http://www.doujinshi.org/browse/author/110758/Mokumokuren/ 110758 符合筆數1_作品筆數11
目目蓮 11 num
========v1
page:1
get
[u'\u3082\u304f\u3082\u304f\u308c\u3093']
[u'\u3082\u304f\u3082\u304f\u308c\u3093', u'\u76ee\u76ee\u84ee']
11 11
1 check 1 http://www.doujinshi.org/book/681603/
#[艦隊これくしょん -艦これ-][サンシャインクリエイション 62]高雄さんと伊19ちゃんとバスタイム!
2 check 1 http://www.doujinshi.org/book/753314/Alice-to-Love-Love-Life!/
#[東方Project][コミックマーケット 87]アリスとラブラブライフ!
3 check 1 http://www.doujinshi.org/book/644037/Alice-to-Patchouli-no-Yoasobi-Time!!/
#[東方Project][コミックマーケット 84]アリスとパチュリーの夜遊びタイム!!
4 check 1 http://www.doujinshi.org/book/668919/Aya-chan-kara-Etchi-na-Service-Shite-Moraeta/
#[東方Project][コミックマーケット 85]文ちゃんからエッチなサービスしてもらえた
5 check 1 http://www.doujinshi.org/book/644035/Deriheru-Jou-Sanae-san-ga-Yatte-Kita!!/
#[東方Project][博麗神社 例大祭 10]デリヘル嬢早苗さんがやってきた!!
6 check 1 http://www.doujinshi.org/book/643998/Deriheru-Jou-Sanae-san-ni-Gohoushi-Shite-Moratta/
#[東方Project][東方紅楼夢 09]デリヘル嬢早苗さんにご奉仕してもらった
7 check 6 http:

''

In [154]:
# -*- coding: utf-8 -*- 
import urllib , requests , sys ,string ,time
from bs4 import BeautifulSoup
from bs4 import SoupStrainer
from time import gmtime, strftime

mlink = 'http://www.doujinshi.org/'
fkey = open('key.txt', 'r')
key = fkey.readline()#key=作者
fkey.close()

key4 = urllib.quote(key)

link="http://www.doujinshi.org/search/item/\
?order=objects&T=author&flow=DESC&sn="+key4+"&Q=s&match=3&page=1"
link="http://www.doujinshi.org/browse/author/42893/Hisasi/?order=title&page=1"
link="http://www.doujinshi.org/browse/author/9312/Ootomo-Takuji/"
#link='http://www.doujinshi.org/browse/author/9312/Ootomo-Takuji/'

cookies = {'PHPSESSID':'qlac2h749s2fpjtgqlnmqbvlo4',
           ' AGE':'18',
           '__utmt':'1',
           '__utma':'221679373.1195927157.1458231980.1458231980.1458231980.1',
           '__utmb':'221679373.6.10.1458231980',
           '__utmc':'221679373',
           '__utmz':'221679373.1458231980.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none)',
           'LANG':'2'
}
'''PHPSESSID=qlac2h749s2fpjtgqlnmqbvlo4; AGE=18; __utmt=1;\
__utma=221679373.1195927157.1458231980.1458231980.1458231980.1;\
__utmb=221679373.6.10.1458231980;\
__utmc=221679373;\
__utmz=221679373.1458231980.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none); LANG=2#'''

res = requests.get(link,cookies=cookies)
#res = requests.get(link)
res.encoding =  res.apparent_encoding#亂碼處理

#print res.text
r= res.text

#<div class="rounded_box">
#<!--rounded_box-->

r1=r[r.find(u'<div class="rounded_box">'):r.find(u'<!--rounded_box-->')+len(u'<!--rounded_box-->')]#+'</td>'#搜尋結果擷取
#r2=r[r.rfind(u'<div class="rounded_box">'):r.rfind(u'<!--rounded_box-->')+len(u'<!--rounded_box-->')]#+'</td>'#搜尋結果擷取
rt='<table border=0 width=\"100%\" class=\'tablefixing\'>'
r2=r[r.rfind(rt)+len(rt):r.rfind('</table>')]#+'</td>'#搜尋結果擷取
r=r1+r2
#only_a_tags = SoupStrainer(id="mu")#縮小檢索範圍
#soup = BeautifulSoup(res.text,"html.parser")#,  parse_only=only_a_tags)
#soup = BeautifulSoup(res.text,"lxml")#,  parse_only=only_a_tags)

#soup = BeautifulSoup(r,"html.parser")#,  parse_only=only_a_tags)
#sou = BeautifulSoup(r2,"lxml")

print 'sample=%E7%8A%AC'
print key4
#print len(res.text),len(str(sou)),len(r)


#全轉半
def Q2B(ustring):
    fs=u'０１２３４５６７８９ＱｑＷｗＥｅＲｒＴｔＹｙＵｕＩｉＯｏＰｐＡａＳｓＤｄＦｆＧｇＨｈＪｊＫｋＬｌＺｚＸｘＣｃＶｖＢｂＮｎＭｍ'
    hs=u'0123456789QqWwEeRrTtYyUuIiOoPpAaSsDdFfGgHhJjKkLlZzXxCcVvBbNnMm'
    rstr = ''
    for tm in ustring:
        if fs.find(tm)+1:
            tm = hs[fs.find(tm)]
        rstr = rstr + tm
    ustring = rstr
    ustring=ustring.lower()
    return ustring

#作品頁面
def finddata(dlink,check,oname):
    res = requests.get(dlink,cookies=cookies)
    res.encoding =  res.apparent_encoding#亂碼處理
    rr= res.text
    #print rr,typr(rr)
    #rtn='<tr>aaa</tr>'
    rtn=rr[rr.find(u'<tr><td><B>イベント:'):rr.find(u'<tr><td><B>シリーズ:')]
    #<tr><td colspan=3><br><div class="list">著者:
    #</div></td></tr>
    #<tr><td colspan=3><br><div class="list">属性:
    #rr=rr[rr.find(u'<tr><td colspan=3><br><div class="list">著者:'):rr.find(u'<tr><td colspan=3><br><div class="list">属性:')]
    rr=rr[rr.find(u'<tr><td colspan=3><br><div class="list">著者:'):]
    #print rr,type(rr)
    #print rr.find(u'<tr><td colspan=3><br><div class="list">',2)
    rr=rr[:rr.find(u'<tr><td colspan=3><br><div class="list">',1)]
    #print rr,type(rr)
    if check==1:
        rr=rtn+'\n'+rr
    #rr=rr[:rr.find(u'属性:</div></td></tr>')]
    #only_a_tags = SoupStrainer(class_="mainborder")#縮小檢索範圍
    so=BeautifulSoup(rr,"lxml")#,  parse_only=only_a_tags)
    #print len(res.text),len(str(so)),len(r)
    #print so
    #print so.select('.rounded_box')[1]
    #print so.select('.mainborder')[0]
    s=0
    
    dcxx=''
    if check==1:
        #print '#d#',so.select('tr')[11],so.select('td')[22]
        dcxx=so.a.text#so.select('tr')[0]
        print 'cxx',dcxx
        s=2
        #e=so.select('tr').index('<tr><td colspan="3"><br/><div class="list">原作:</div></td></tr>')
    
    #print so.select('.list')[0]#.parent.parent
    dname=''
    n=0
    dcheck=0
    doname=''
    for b in so.select('td')[1+s::3]:#[8:16]
        #print 'B:',so.select('td').index(b),b.text.strip().strip('\n').replace('\t','').replace('\n','')#.split('\n')
        if Q2B(b.text).strip().strip('\n').replace('\t','').replace('\n','')==key.decode('utf8'):
            dcheck=1
        #elif (len(oname)>0) and (Q2B(b.text).strip().strip('\n').replace('\t','').replace('\n','') in oname):
            #doname=Q2B(b.text).strip().strip('\n').replace('\t','').replace('\n','')
        dname=dname+Q2B(b.text).strip().strip('\n').replace('\t','').replace('\n','')+'_'
        n=n+1
        #if so.select('td').index(b) >12:
            #break
    print dname,n
    if (n>1) and (check != 4):
        check=5
        if check==1:
            check=2
    if dcheck==0:
        check=7
        if check==2 or check==1:
            check=6
    #name=
    #cxx
    
    #book
    #club
    #raw
    return dname,dcxx,check#,doname,n
    ########
    for d in sou.select('.m-boxDetailProductInfoMainList__description__list')[0].select('a'):
        if d.text==key.decode('utf8'):
            dcheck=1
        dname=dname+'_'+d.text
    dname=dname.strip(u'_')
    dbook=sou.select('.m-boxDetailProductInfoMainList__description__list')[1].text.strip().strip(u'\n').strip(u'\t')
    if dcheck==1 :
        if key.decode('utf8') != dname:
            dcheck=3
    
    return dname,dbook,dcheck

bn=r2.count('\"thumbnail\"')
doc=r2.split('<!--round_middle-->')
#print doc[50]
doc.pop()
print bn,len(doc)#,doc[0],doc[50]
a=0
for tm in doc[:]:
#for tm in soup.select('.thumbnail'):
    #print a,tm
    #print soup.select('.bookinfo')[a]
    #sou = BeautifulSoup(str(tm)+str(soup.select('.bookinfo')[a]),"lxml")
    sou = BeautifulSoup(tm,"lxml")
    #print a,sou.select('.thumbnail')[0]
    #clink=n.select('.m-boxListBookProductTmb')[0].select('a')[0].get('href')
    
    cbook=sou.select('.tab.LPEXACT1')[0].text
    cclub=sou.select('.tab.LPEXACT1')[2].text
    craw=sou.select('.tab.LPEXACT1')[4].text
    ctype=sou.select('.tab.LPEXACT1')[5].text
    
    clink=sou.a.get('href')
    cdata=clink[6:]
    cdata=cdata[:cdata.find('/')]
    print cdata,cbook,cclub,craw,ctype#,clink
    blink=mlink+clink[1:]
    print blink
    check=0
    '''
同人CG_同人ノベル_同人誌
漫画
雑誌
商業ノベル_商業本その他

1單同_同人CG_同人ノベル_同人誌
2多同

3單漫_漫画
4雑誌
5多漫_商業ノベル_商業本その他

6別名同
7別商
    #'''
    oname=['睦茸','三色網戸']
    #類型
    if (u'同人CG' in ctype) or (u'同人ノベル' in ctype) or (u'同人誌' in ctype):
        check=1
    elif u'漫画' in ctype:
        check=3
    elif u'雑誌' in ctype:
        check=4
    elif (u'商業ノベル' in ctype) or (u'商業本その他' in ctype):
        check=5
    else:
        a=a+1
        print 'X'
        continue
    dname,dcxx,check=finddata(blink,check,oname)#連結_別名_判斷類型
    print 'check:',check
    #cname
    #
    a=a+1
    #break
    #if a>1:
    if a==bn:
        break


sample=%E7%8A%AC
%E5%A4%A7%E5%8F%8B%E5%8D%93%E4%BA%8C
50 50
449516 メテオさんストライクです!番外編 HARDMOJAH オリジナル 同人誌
http://www.doujinshi.org/book/449516/
cxx 不詳
たくじ_ 1
check: 7
515846 伊織でIN! Number2 アイドルマスター 同人誌
http://www.doujinshi.org/book/515846/
cxx コミックマーケット 81
たくじ_ 1
check: 7
541496 スルメちゃん(仮) HARD MOJAH オリジナル 同人誌
http://www.doujinshi.org/book/541496/
cxx コミックマーケット 82
春日旬_たくじ_ 2
check: 7
567507 メテオさんストライクです! 2 (none) (none) 漫画
http://www.doujinshi.org/book/567507/
たくじ_ 1
check: 7
568864 メテオさんストライクです! 3 (none) (none) 漫画
http://www.doujinshi.org/book/568864/
たくじ_ 1
check: 7
597808 ブラコンアンソロジー Liqueur -リキュール- 2 (none) オリジナル 漫画
http://www.doujinshi.org/book/597808/
渡辺静_よしだもろへ_雨隠ギド_壱号_来瀬ナオ_藤屋いずこ_キダニエル_袴田めら_昼沖太_位置原光z_五十嵐電マ_伊咲ウタ_狩野蒼穹_kashmir_風華チルヲ_カザマアヤミ_kima-gray_草野紅壱_巻田佳春_みりの_大朋めがね_大石中二_桜沢鈴_たくじ_ 24
check: 7
645857 はじぷる! 1 (none) (none) 漫画
http://www.doujinshi.org/book/645857/
たくじ_ 1
check: 7
672645 アイドル100%膣内ライブ! Number2 ラブライブ! School idol project 同人誌
http://www.doujinshi.org/book/672645/
cxx コミックマ

In [228]:
a=['ccc','ddd']
b=['1','aa','cc','v','ddd','dd']
for c in b:
    print c,
    if c in a:
        print 'get',
    print '.'
x=u'屍'
y='凹'
#x=y

print type(x),type(y),
print x.encode('utf8'),
y=y.encode('utf8')
#print str(x),
print str(y)
#print x+y
#print y+x


 1 .
aa .
cc .
v .
ddd get .
dd .
<type 'unicode'> <type 'str'> 屍

UnicodeDecodeError: 'ascii' codec can't decode byte 0xe5 in position 0: ordinal not in range(128)